<a href="https://colab.research.google.com/github/kohgeonbu/koh-git/blob/main/kohgeonbu_report(opencv).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install mediapipe


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.1/36.1 MB 43.5 MB/s eta 0:00:00


In [22]:
!ffmpeg -i "/content/drive/My Drive/smh.mp4" -vcodec libx264 -acodec aac "/content/drive/My Drive/smh_converted.mp4"


ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [23]:
cap = cv2.VideoCapture("/content/drive/My Drive/smh_converted.mp4")

if not cap.isOpened():
    print("동영상을 열 수 없습니다.")
else:
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"FPS: {fps}, 총 프레임 수: {frame_count}, 길이(초): {frame_count / fps if fps > 0 else '계산 불가'}")
cap.release()


FPS: 29.97002997002997, 총 프레임 수: 412, 길이(초): 13.747066666666667


In [45]:
import cv2
import mediapipe as mp
from google.colab import drive
import shutil
import numpy as np

# 1. 구글 드라이브 마운트
drive.mount('/content/drive', force_remount=True)

# 2. Mediapipe 설정
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=2,
    min_detection_confidence=0.8,  # 신뢰도 향상
    min_tracking_confidence=0.9   # 추적 신뢰도 향상
)
mp_drawing = mp.solutions.drawing_utils

# 안정화 변수 초기화
prev_landmarks = None  # 이전 프레임의 랜드마크 저장
recent_landmarks = []  # 최근 프레임의 랜드마크 누적
alpha = 0.7  # 안정화 강도
max_delta_threshold = 0.1  # 큰 변화 감지 임계값
fallback_frames = 5  # 큰 변화 후 몇 프레임 동안 보완 적용
fallback_counter = 0  # 대체 보완 프레임 카운터

# 3. 입력 영상 불러오기
input_video_path = '/content/drive/My Drive/smh_converted.mp4'
cap = cv2.VideoCapture(input_video_path)

# 원본 영상 정보
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
original_fps = int(cap.get(cv2.CAP_PROP_FPS))
out = cv2.VideoWriter('processed_smh.mp4', cv2.VideoWriter_fourcc(*'mp4v'), original_fps, (frame_width, frame_height))

# 5. 동영상 처리
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # (1) 좌우 반전
    frame = cv2.flip(frame, 1)

    # (2) RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # (3) 손 검출
    results = hands.process(rgb_frame)

    # (4) 손끝 랜드마크 안정화 및 표시
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            current_landmarks = np.array([[lm.x, lm.y, lm.z] for lm in hand_landmarks.landmark])
            recent_landmarks.append(current_landmarks)

            # 최근 5개 프레임 유지
            if len(recent_landmarks) > 5:
                recent_landmarks.pop(0)

            # 큰 변화 감지 및 보완
            if prev_landmarks is None:
                # 첫 프레임 초기화
                prev_landmarks = current_landmarks
                stabilized_landmarks = current_landmarks  # 안정화 없이 현재 값 사용
            else:
                delta = np.linalg.norm(prev_landmarks - current_landmarks, axis=1)

                if np.any(delta > max_delta_threshold):  # 큰 변화 감지
                    print("큰 변화 감지, 보완 적용")
                    fallback_counter = fallback_frames  # 보완 카운터 초기화

                if fallback_counter > 0:
                    # 최근 5개 프레임 평균으로 대체
                    stabilized_landmarks = np.mean(recent_landmarks, axis=0)
                    fallback_counter -= 1  # 카운터 감소
                else:
                    # 가중 평균 안정화
                    stabilized_landmarks = alpha * prev_landmarks + (1 - alpha) * current_landmarks

            prev_landmarks = stabilized_landmarks  # 안정화된 값 업데이트

            # 손가락 끝 랜드마크 정의 (엄지 제외)
            fingertip_indices = [
                mp_hands.HandLandmark.INDEX_FINGER_TIP,
                mp_hands.HandLandmark.MIDDLE_FINGER_TIP,
                mp_hands.HandLandmark.RING_FINGER_TIP,
                mp_hands.HandLandmark.PINKY_TIP
            ]

            # 각 손가락 끝에 점 그리기
            for fingertip in fingertip_indices:
                point = stabilized_landmarks[fingertip.value]
                x, y = int(point[0] * frame.shape[1]), int(point[1] * frame.shape[0])
                cv2.circle(frame, (x, y), 8, (0, 0, 255), -1)

    # (5) 처리된 프레임 저장
    out.write(frame)

# 6. 자원 해제
cap.release()
out.release()

# 7. 결과를 구글 드라이브로 이동
shutil.move('processed_smh.mp4', '/content/drive/My Drive/processed_smh.mp4')
print("처리된 영상이 구글 드라이브에 저장되었습니다.")


Mounted at /content/drive
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
큰 변화 감지, 보완 적용
처리된 영상이 구글 드라이브에 저장되었습니다.


In [46]:
!git clone https://github.com/kohgeonbu/koh-git.git

Cloning into 'koh-git'...
remote: Enumerating objects: 17, done.
remote: Counting objects: 100% (17/17), done.
remote: Compressing objects: 100% (14/14), done.
remote: Total 17 (delta 3), reused 5 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (17/17), 91.95 KiB | 879.00 KiB/s, done.
Resolving deltas: 100% (3/3), done.


In [47]:
!git add .
!git commit -m "kohgeonbu report(opencv)"
!git push origin main

fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
fatal: not a git repository (or any of the parent directories): .git
